# El visualizador de root files

In [2]:
conda install -c conda-forge root


done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 

## Merge Root Files

In [1]:
import ROOT
import os

def merge_files(directory, starts_with, output_name):

    for file in os.listdir(directory):
        
        if file.endswith('.root') and not file.startswith('merge'):
            if not starts_with == '':
                if file.startswith(starts_with):
                    file_list = [os.path.join(directory, file)]
            else:
                file_list = [os.path.join(directory, file)]
    
    merged_file = os.path.join(directory, output_name)
    counter = 0
    while True:
        if not os.path.exists(f"{merged_file}{counter}.root"):
            merged_file = f"{merged_file}{counter}.root"
            break
        counter = counter + 1

    merger = ROOT.TFileMerger()
    merger.OutputFile(merged_file)
    
    for i in file_list:
        merger.AddFile(i)
    merger.Merge()

    print("Merged files into", merged_file)

ModuleNotFoundError: No module named 'ROOT'

In [ ]:
directory = 'build/root'
starts_with = 'root'
output_name = 'merged'

merge_files(directory, starts_with, output_name)

## Draw Graph from Merge.root

In [ ]:
import ROOT
import os

def scatterplot_from_root(directory, input_file, tree_name, x_branch, y_branch, title, x_label, y_label, output_image):

    input_file = os.path.join(directory, input_file)
    
    file = ROOT.TFile(input_file, "READ")
    tree = file.Get(tree_name)
    
    canvas = ROOT.TCanvas("canvas", "X vs Y Scatter Plot", 800, 600)
    graph = ROOT.TGraph()
    
    points = []
    for event in tree:
        x = getattr(event, x_branch)
        y = getattr(event, y_branch)
        points.append((x, y))

    # Sort points based on x-values
    points.sort(key=lambda p: p[0])

    # Create a TGraph and add sorted points
    graph = ROOT.TGraph()
    for i, (x, y) in enumerate(points):
        graph.SetPoint(i, x, y)

    graph.SetTitle(title)
    graph.GetXaxis().SetTitle(x_label)
    graph.GetYaxis().SetTitle(y_label)

    graph.SetMarkerStyle(20)  # Circular marker
    graph.SetMarkerSize(0.4)
    graph.SetLineColor(ROOT.kBlue)
    graph.SetLineWidth(2)

    graph.Draw("AP")  # "A" for axes, "P" for points, "L" for line
    ROOT.gStyle.SetImageScaling(2.0)
    canvas.SaveAs(output_image + '.jpg')
    file.Close()
    print(' ===== Scatter plot saved as', output_image, '=======')

In [ ]:
directory = 'build/root'
file_name = 'merged4.root'

tree_name = "Photons"
x_branch = "X_axis"
y_branch = "Y_axis"

title = "title"
x_label = "x axis"
y_label = "y axis"

output_image = "xy_plot"

scatterplot_from_root(directory, file_name, tree_name, x_branch, y_branch, title, x_label, y_label, output_image)

## Extraer .root -> Archivo CSV

In [ ]:
import ROOT
import os
import csv

def copy_data_csv(directory, root_name_starts, tree_name, x_branch, y_branch, z_branch, csv_save_as):

    csv_file  = os.path.join(directory, csv_save_as)
    
    with open(csv_file, 'w', newline='') as file:

        writer = csv.writer(file)
        writer.writerow(['File', tree_name, x_branch, y_branch, z_branch])
        
        for file_name in sorted(os.listdir(directory)):
            if file_name.endswith('.root') and file_name.startswith(root_name_starts):

                file_path = os.path.join(directory, file_name)

                try:
                    
                    root_file = ROOT.TFile(file_path)

                    tree = root_file.Get(tree_name)
                    if not tree:
                        print(f"Tree '{tree_name}' not found in {file_path}")
                        continue

                    branch1 = tree.GetBranch(x_branch)
                    if not branch1:
                        print(f"Branch1 '{x_branch}' not found in tree '{tree_name}' in {file_path}")
                        continue

                    branch2 = tree.GetBranch(y_branch)
                    if not branch2:
                        print(f"Branch2 '{y_branch}' not found in tree '{tree_name}' in {file_path}")

                    branch3 = tree.GetBranch(z_branch)
                    if not branch3:
                        print(f"Branch3 '{z_branch}' not found in tree '{tree_name}' in {file_path}")

                    # if not branch3:
                    #     if not branch2:
                    #         writer.writerow([file_name, tree_name, x_branch])
                    #     else:
                    #         writer.writerow([file_name, tree_name, x_branch, y_branch])
                    # else:
                    #     writer.writerow([file_name, tree_name, x_branch, y_branch, z_branch])


                    entries1 = branch1.GetEntries()
                    print('Número de entradas:', entries1)

                    for entry in range(entries1):
                        
                        tree.GetEntry(entry)
                        value1 = getattr(tree, x_branch)

                        if not branch3:
                            if not branch2:
                                writer.writerow([file_name, tree_name, value1])
                            else:
                                value2 = getattr(tree, y_branch)
                                writer.writerow([file_name, tree_name, value1, value2])
                        else:
                            value2 = getattr(tree, y_branch)
                            value3 = getattr(tree, z_branch)
                            writer.writerow([file_name, tree_name, value1, value2, value3])
                        
                    root_file.Close()

                except Exception as e:
                    print(f"Error procesando {file_path}: {e}")
                
    print('CSV file created as:', csv_file)

In [ ]:
directory = 'build/root/'
root_name_starts = "merged4"

tree_name = "Photons"
x_branch  = "X_axis"
y_branch  = "Y_axis"
z_branch  = "z_branch"

csv_save_as = 'test2' + '.csv'

copy_data_csv(directory, root_name_starts, tree_name, x_branch, y_branch, z_branch, csv_save_as)

## Plot CSV (X axis vs. Y axis)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_xy(directory, csv_name, x_branch, y_branch, title, x_label, y_label):

    csv_file = os.path.join(directory, csv_name)
    df = pd.read_csv(csv_file)

    x = df[x_branch]
    y = df[y_branch]

    SIZE_DEFAULT = 14
    SIZE_LARGE = 20

    plt.rc("font", family = 'Century Expanded')  
    plt.rc("font", weight = "normal")  
    plt.rc("font",  size      = SIZE_DEFAULT)  
    plt.rc("axes",  titlesize = SIZE_LARGE + 2)  
    plt.rc("axes",  labelsize = SIZE_LARGE)  
    plt.rc("xtick", labelsize = SIZE_DEFAULT)  
    plt.rc("ytick", labelsize = SIZE_DEFAULT)  

    plt.figure(figsize = (10, 6))
    plt.tight_layout(rect = [0, 0.01, 1, 1])

    plt.plot(x, y, marker = 'o')
    plt.grid(True, alpha = 0.5)
    
    plt.xlabel(x_label, fontweight = "bold", labelpad = 21)
    plt.ylabel(y_label, fontweight = "bold", labelpad = 22)
    plt.title (title  , fontweight = "bold", pad = 25)
    
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['top'].set_alpha(0.3)
    ax.spines['right'].set_alpha(0.3)
    ax.spines['bottom'].set_alpha(0.7)
    ax.spines['left'].set_alpha(0.7)

    plt.savefig(title + '.png', dpi = 10)
    plt.show()

    print('Plot saved as', title + '.png')

In [ ]:
directory = 'build/root/'
csv_name = 'test2.csv'

x_branch = "X_axis"
y_branch = 'Y_axis'

title = "title"
x_label = r"x axis ($keV$)"
y_label = r"y axis"

plot_xy(directory, csv_name, x_branch, y_branch, title, x_label, y_label)

## Plot CSV Histogram

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def histogram(directory, csv_name, data, set_bins, positive, title, x_label, y_label):

    csv_file = os.path.join(directory, csv_name)
    df = pd.read_csv(csv_file)

    points = df[data]
    if positive == True:
        points = df[df[data] > 0]

    plt.figure(figsize=(10, 7))
    #plt.tight_layout(rect=[0, 1, 1, 1])

    SIZE_DEFAULT = 14
    SIZE_LARGE = 20

    plt.rc("font", family = 'Century Expanded')  
    plt.rc("font", weight = "normal")  
    plt.rc("font",  size      = SIZE_DEFAULT)  
    plt.rc("axes",  titlesize = SIZE_LARGE + 2)  
    plt.rc("axes",  labelsize = SIZE_LARGE)  
    plt.rc("xtick", labelsize = SIZE_DEFAULT)  
    plt.rc("ytick", labelsize = SIZE_DEFAULT)  

    plt.hist(points, bins = set_bins, alpha = 1)

    plt.title(title, pad = 15)
    plt.xlabel(x_label, labelpad = 15)
    plt.ylabel(y_label, labelpad = 15)
    plt.grid(True)

    plt.savefig(title + '.png', dpi = 300)
    plt.show()

    print('Plot saved as', title + '.png')

In [ ]:
directory = 'build/root/'
csv_name = 'test2.csv'

data = "X_axis"
set_bins = 50
positive = False

title = 'title'
x_label = 'x axis (keV)'
y_label = 'Frequency'

histogram(directory, csv_name, data, set_bins, positive, title, x_label, y_label)